In [1]:
import pandas as pd
import numpy as np
import re

df = pd.read_parquet('output/sentiments.parquet')

# Dataset cleaning
df.dropna(inplace=True)
df = df[(df['review'].apply(lambda x: re.match(r'^[()\[\]{}\'!,.: ]+$', x) is None)) & (df['review'].apply(lambda x: len(x) > 1))]

df['sentiment'] = df['sentiment'].map({
    'Very Negative': -2,
    'Negative': -1,
    'Neutral': 0,
    'Positive': 1,
    'Very Positive': 2
})

#df.rename(columns={'game': 'SteamID'}, inplace=True)
#df = games[['game', 'SteamID']].merge(df, on='SteamID', how='inner').drop(columns=['SteamID'])
df['game'] = df['game'].astype('category')

df

,topic,review,embedding,sentiment,game
0,gameplay mechanics,Excellent game mechanics hampered by really sh...,"[0.02064609, 0.058051355, -0.15955217, -0.0282...",-1,1653100
1,gameplay mechanics,I love the narrator just talking in the backgr...,"[-0.01935661, 0.021597555, -0.18828672, -0.039...",1,1653100
2,gameplay mechanics,The game has received frequent and substantial...,"[0.030498626, 0.035088323, -0.17761268, -0.017...",1,1653100
3,gameplay mechanics,Heliotropism is a roguelike Tree simulator wit...,"[-0.005617, 0.06219709, -0.17552276, -0.042904...",1,1653100
4,narrator and audio,"the narrator is cool at first, then hyper anno...","[0.01688186, 0.010326986, -0.16718766, -0.0392...",-1,1653100
...,...,...,...,...,...
96560,music and sound,Хорошее музыкальное и звуковое сопровождение.,"[-0.035658363, -0.0012977426, -0.1295827, 0.02...",1,2300300
96561,music and sound,"I like the music, it's nice relaxing.","[-0.020432916, 0.03288259, -0.18042335, -0.001...",1,2300300
96562,music and sound,The music is nice relaxing.,"[-0.0041316957, 0.0568026, -0.18956739, 0.0036...",1,2300300
96563,price and value,The asking price might be a bit steep for what...,"[0.012826282, 0.04906256, -0.20521845, 0.02795...",0,2300300


In [2]:
import os
from ollama import Client
from src.embed import get_embedding, NomicPrefix
from sklearn.metrics.pairwise import cosine_similarity

OLLAMA = Client(
    host=f"http://{os.environ.get('OLLAMA_HOST', 'localhost')}:{os.environ.get('OLLAMA_PORT', '11434')}",
)

def get_similars(queries, embeddings, threshold):    
    query_embedding = get_embedding(queries)
    similarities = cosine_similarity(query_embedding, embeddings)
    
    indices = pd.DataFrame(data=np.argwhere(similarities > threshold), columns=['query', 'review'])
    indices['query'] = indices['query'].map(lambda x: queries[x])

    return indices.groupby('query')['review'].apply(list).to_dict()

In [3]:
citations_df = df.drop_duplicates(subset='review')
citation_embeddings = np.vstack(citations_df['embedding'].values)

#all_topics = df['topic'].unique()
#topics_embeddings = get_embedding(list(all_topics))
#pd.DataFrame(data=topics_embeddings, index=all_topics).to_parquet('output/topics.parquet')

all_topics = pd.read_parquet('output/topics.parquet')
topics_embeddings = all_topics.values
all_topics = all_topics.index

In [4]:
len(all_topics)

6449

In [5]:
#from yellowbrick.cluster import KElbowVisualizer
#from sklearn.cluster import KMeans
#import seaborn as sns
#import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE

#plt.set_loglevel('error')

#model = KMeans()
#visualizer = KElbowVisualizer(model, k=range(5, 60, 2))

#topics_clustering_embeddings = get_embedding(list(all_topics), prefix=NomicPrefix.CLUSTERING)

#visualizer.fit(topics_clustering_embeddings)
#visualizer.show()

In [6]:
#kmeans = KMeans(n_clusters=visualizer.elbow_value_).fit_predict(topics_clustering_embeddings)
#tsne = TSNE(n_components=2, verbose=1, max_iter=500).fit_transform(topics_clustering_embeddings)

In [7]:
# Select 10 random clusters
#df_cluster = pd.DataFrame(data=tsne, columns=['x', 'y'])
#df_cluster['cluster'] = kmeans

#random_clusters = np.random.choice(np.unique(kmeans), 10)
#df_cluster = df_cluster[df_cluster['cluster'].isin([2, 5, 7, 8, 9, 14, 15, 25, 28, 30])]

#sns.scatterplot(data=df_cluster, x='x', y='y', hue='cluster', palette='tab10', s=30, legend=False)
#plt.savefig('output/out/clusters.pdf', bbox_inches='tight', format='pdf')
#plt.show()

In [8]:
def search_similars(queries):
    similars_by_text = get_similars(queries, citation_embeddings, 0.8)
    similars_by_topic = get_similars(queries, topics_embeddings, 0.85)

    results = {}
    for query in queries:
        sim_a = citations_df.iloc[similars_by_text.get(query, [])]

        included_topics = all_topics[similars_by_topic.get(query, [])]
        sim_b = df[df['topic'].isin(included_topics)]

        # Merge and drop duplicates
        sim = pd.concat([sim_a, sim_b]).drop_duplicates(subset='review')
        results[query] = sim[['review', 'sentiment', 'game']]
    
    return results

In [9]:
import json

queries: dict = json.load(open('data/queries.json'))

In [10]:
results = [
    search_similars(q)
    for _, q in queries.items()
]

In [11]:
results_df = pd.concat([
    pd.concat([d.assign(query=q) for q, d in cat_q.items()])
    for cat_q in results
])

results_df['query'] = results_df['query'].astype('category')
results_df

,review,sentiment,game,query
1191,Nice puzzle game,1,1875060,Puzzle
1315,I like puzzles.,1,1875060,Puzzle
1329,Nice puzzle game.,1,1875060,Puzzle
1391,fun puzzle game,1,1875060,Puzzle
1688,Lovely little puzzle game,1,1875060,Puzzle
...,...,...,...,...
94157,"O jogo é muito bonitinho, o conceito é legal, ...",1,222730,Educational Value
94158,"Eu daria pro meu filho ou filha jogar, se eu t...",1,222730,Educational Value
96107,I could easily see this being the kind of educ...,1,2289390,Educational Value
96108,An adorable management game with an ecological...,1,2289390,Educational Value


In [12]:
results_df.groupby('query', observed=True).agg({
    'sentiment': ['mean', 'std', 'count']
}).round(2).sort_values(('sentiment', 'mean'), ascending=False)

sentiment            
                                       mean   std count
query                                                  
Enjoyment and Fun                      1.55  0.67  4413
Relaxation and Calmness                1.42  0.66  3730
Immersive Experience                   1.34  0.70    68
Atmosphere and Ambiance                1.29  0.84   705
Realism                                1.22  0.98  2080
Educational Value                      1.21  0.83   469
Sound and Music                        1.02  0.99  1090
Character customization                1.01  0.86   129
Simulation                             0.99  1.07   369
Graphics and Aesthetics                0.96  1.12  3476
Exploration and Discovery              0.94  0.82   135
Sandbox                                0.90  0.77   104
Storytelling and Narrative             0.87  1.11   382
Map customization                      0.85  0.91   100
Puzzle                                 0.84  0.88   288
Multiplayer                            0.68  1.07  2374
Replayability                          0.53  1.08  1746
Emotional Connection                   0.40  1.41   901
Survival                               0.09  1.27   131
Achievements and Rewards               0.02  1.03    55
Game length                           -0.13  1.00  1239
Learning curve and Difficulty         -0.19  1.03   845
Game Performance and Optimization     -0.53  1.18   776
Controls and Input                    -0.59  0.94   135
Game pacing                           -0.61  1.10   197

In [13]:
def get_table(results, agg_fn):
    df = pd.DataFrame()
    for queries in results:
        for query, citations in queries.items():
            #df[(category, query)] = agg_fn(citations)
            df[query] = agg_fn(citations)
    #df.columns = pd.MultiIndex.from_tuples(df.columns, names=['category', 'query'])
    return df

counts = get_table(results, lambda x: x['game'].value_counts()).sort_index()
means = get_table(results, lambda x: x.groupby('game', observed=False)['sentiment'].mean()).sort_index()

# Select games with more than 10 hits in almost 2 categories
selected_games = counts[counts[counts >= 10].count(axis=1) > 1].index

# Hide values between -0.5 and 0.5 and mean support less than 10
means = means[(means.abs() > 0.5) & (counts >= 10)].loc[selected_games].dropna(how='all', axis=1).dropna(how='all', axis=0)

means

,Puzzle,Survival,Sandbox,Simulation,Storytelling and Narrative,Learning curve and Difficulty,Game pacing,Enjoyment and Fun,Relaxation and Calmness,Immersive Experience,...,Realism,Exploration and Discovery,Atmosphere and Ambiance,Graphics and Aesthetics,Sound and Music,Game length,Game Performance and Optimization,Controls and Input,Multiplayer,Educational Value
game,,,,,,,,,,,,,,,,,,,,,
1028590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.636364,NaN,NaN,...,0.720000,NaN,NaN,1.000000,NaN,NaN,-1.545455,NaN,NaN,NaN
1062090,NaN,NaN,0.843750,1.305882,NaN,NaN,-0.844828,1.563977,1.343648,NaN,...,NaN,NaN,1.291667,0.897260,0.917910,NaN,NaN,NaN,NaN,NaN
1133120,NaN,NaN,NaN,0.750000,NaN,NaN,NaN,1.476190,1.250000,NaN,...,-0.900000,NaN,NaN,-0.941176,NaN,NaN,-1.100000,NaN,NaN,NaN
1372320,0.833333,NaN,0.608696,NaN,NaN,NaN,NaN,1.476190,1.481884,NaN,...,NaN,NaN,1.169811,1.261745,1.300000,NaN,NaN,NaN,NaN,NaN
1390190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.641304,1.523256,NaN,...,NaN,NaN,1.458333,1.250000,0.916667,NaN,NaN,NaN,NaN,0.777778
1539580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.971429,1.013889,NaN,...,NaN,NaN,1.400000,NaN,NaN,NaN,-1.357143,NaN,NaN,NaN
1575510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.333333,NaN,...,NaN,NaN,NaN,1.363636,NaN,NaN,NaN,NaN,NaN,NaN
1593030,0.789474,NaN,NaN,NaN,NaN,NaN,NaN,1.431818,1.445769,NaN,...,NaN,NaN,1.420000,1.364706,1.216867,NaN,NaN,NaN,NaN,1.266667
1837330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.166667,1.235294,NaN,NaN,NaN,NaN,NaN


In [14]:
games = pd.read_csv('./data/Game List - Final.csv')
games = games.dropna(subset=['SteamID']).rename(columns={'List (merge)': 'game'})
games = games[games['Final decision'] == 'Yes']

games['SteamID'] = games['SteamID'].astype(int).astype(str)
games.set_index('SteamID', inplace=True)
games = games[['game', 'LongCode']]
games.rename(columns={'LongCode': 'code'}, inplace=True)
games = games.loc[selected_games]

id_code_map = games['code'].to_dict()

games

,game,code
game,,
1028590,The Wilds,Wilds
1062090,Timberborn,Timberborn
1133120,Ecosystem,Ecosys
1372320,Cloud Gardens,CloudG
1390190,APICO,Apico
1539580,Bird,Bird
1575510,Seedlings,Seedlings
1593030,Terra Nil,TerraNil
1837330,Gibbon: Beyond the Trees,Gibbon


In [15]:
top_games = df.groupby('game', observed=True).size().sort_values(ascending=False)
top_games.index = top_games.index.map(id_code_map)

top_games[~top_games.index.isna()]

game
Timberborn    29138
Eco           12069
Wolf-AE        9860
TerraNil       7352
Reus           5470
Meadow         4937
Shelter2       3510
Shelter        3363
Equi           3287
CloudG         2592
Endling        2425
Wolf-1         1645
Apico          1625
Reus2          1582
BeeSim         1098
Bird            777
Paws            712
Gibbon          492
Ecosys          459
Wilds           424
Paperbark       409
Preserve        403
DNectar         270
Growth          246
UPenguin        188
Seedlings        92
dtype: int64

In [16]:
from itertools import combinations

results_df = results_df[results_df['game'].isin(selected_games)].copy()
results_df['game'] = results_df['game'].map(id_code_map)
grouped = results_df.groupby('query', observed=True)
cooccurrence = {}

# Count co-occurrences of reviews between query pairs
for (query1, reviews1), (query2, reviews2) in combinations(grouped, 2):
    shared_reviews = pd.merge(reviews1, reviews2, on='review', how='inner')
    cooccurrence[(query2, query1)] = shared_reviews

cooccurrence = pd.DataFrame([
    {'query1': q1, 'query2': q2, 'shared_reviews': count}
    for (q1, q2), count in cooccurrence.items()
])

cooccurrence = cooccurrence[cooccurrence['shared_reviews'].apply(lambda x: len(x) >= 10)]

cooccurrence['n'] = cooccurrence['shared_reviews'].apply(len)
cooccurrence['mean'] = cooccurrence['shared_reviews'].apply(lambda x: np.mean(x['sentiment_x']))
cooccurrence['std'] = cooccurrence['shared_reviews'].apply(lambda x: np.std(x['sentiment_x']))
cooccurrence['top-3'] = cooccurrence['shared_reviews'].apply(lambda x: x.groupby('game_x', observed=True)['sentiment_x'].mean().nlargest(3).index.tolist())

cooccurrence

,query1,query2,shared_reviews,n,mean,std,top-3
33,Graphics and Aesthetics,Atmosphere and Ambiance,...,245,1.310204,0.788759,"[Bird, Equi, Endling]"
40,Relaxation and Calmness,Atmosphere and Ambiance,...,56,1.535714,0.596546,"[Equi, CloudG, Reus2]"
58,Map customization,Character customization,...,52,1.076923,0.729756,"[BeeSim, Timberborn, Wolf-AE]"
91,Enjoyment and Fun,Educational Value,...,14,1.785714,0.410326,"[Wolf-AE, Wolf-1]"
102,Realism,Educational Value,...,15,1.600000,0.800000,"[Wolf-AE, BeeSim]"
133,Graphics and Aesthetics,Enjoyment and Fun,...,17,1.705882,0.455645,"[Wolf-1, Timberborn, Wolf-AE]"
137,Multiplayer,Enjoyment and Fun,...,18,1.444444,0.598352,"[Wolf-1, Eco, Wolf-AE]"
139,Realism,Enjoyment and Fun,...,34,1.823529,0.381220,"[Wolf-AE, Wolf-1, BeeSim]"
140,Relaxation and Calmness,Enjoyment and Fun,...,1051,1.437678,0.659172,"[Wolf-AE, Equi, Apico]"
180,Game pacing,Game length,...,43,-0.093023,1.137397,"[Endling, Paperbark, Reus]"


In [17]:
top_3 = pd.DataFrame(
    index=pd.MultiIndex.from_frame(cooccurrence[['query1', 'query2']]),
    columns=['1', '2', '3'],
    data=cooccurrence['top-3'].tolist()
).fillna('').sort_index()

top_3.style.to_latex(
    buf='output/out/top-3.tex',
    position='t',
    position_float='centering',
    caption='Top 3 games with highest sentiment for each query pair',
    label='tab:top-3',
    hrules=True,
)

top_3

ImportError: Missing optional dependency 'Jinja2'. DataFrame.style requires jinja2. Use pip or conda to install Jinja2.

In [ ]:
co_occurred_queries = np.unique(cooccurrence['query1'].tolist() + cooccurrence['query2'].tolist())
co_occurred_queries_dtype = pd.CategoricalDtype(categories=co_occurred_queries, ordered=False)

cooccurrence['query1'] = cooccurrence['query1'].astype(co_occurred_queries_dtype)
cooccurrence['query2'] = cooccurrence['query2'].astype(co_occurred_queries_dtype)

In [ ]:
n_cooc = cooccurrence.pivot_table(index='query1', columns='query2', values='n', aggfunc='sum', observed=False)
n_cooc.sort_index(inplace=True)

# Substitute 0 values with NaN
n_cooc = n_cooc.mask(n_cooc == 0)

mean_cooc = cooccurrence.pivot_table(index='query1', columns='query2', values='mean', aggfunc='sum', observed=False)
mean_cooc.sort_index(inplace=True)
mean_cooc = mean_cooc.mask(n_cooc.isna())

mean_cooc

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 8))
ax = sns.heatmap(
    n_cooc.dropna(axis=1, how='all').dropna(axis=0, how='all'),
    annot=True,
    cmap='Spectral_r',
    fmt='.0f',
    center=0,
)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')

plt.xlabel('Query 1')
plt.ylabel('Query 2')

os.makedirs('output/out', exist_ok=True)
plt.savefig('output/out/cooccurrence.pdf', bbox_inches='tight', format='pdf')
plt.show()


In [ ]:
plt.figure(figsize=(10, 8))

ax = sns.heatmap(
    mean_cooc.dropna(axis=1, how='all').dropna(axis=0, how='all'),
    annot=True,
    fmt='.2f',
    robust=True,
    cmap='Spectral_r',
    center=0,
    vmin=-2,
    vmax=2,
)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')

plt.xlabel('Query 1')
plt.ylabel('Query 2')
plt.savefig('output/out/cooccurrence-mean.pdf', bbox_inches='tight', format='pdf')
plt.show()

In [ ]:
means.index = means.index.map(id_code_map)

# Sort columns by non-null values
means = means[means.count().sort_values(ascending=True).index]

# Sort index by non-null values
means = means.loc[means.count(axis=1).sort_values(ascending=False).index]
means = means.T
means['AVG (all)'] = means.mean(axis=1)

# Move AVG to the first column
cols = means.columns.tolist()
cols = cols[-1:] + cols[:-1]
means = means[cols]

plt.figure(figsize=(15, 10))
sns.heatmap(means, annot=True, cmap='Spectral_r', fmt='.1f', center=0, vmin=-2, vmax=2)
plt.xlabel('Game')
plt.ylabel('Query')
plt.savefig('output/out/mean-sentiment.pdf', format='pdf', bbox_inches='tight')

plt.show()